In [189]:
import os
import numpy as np
import pandas as pd
import sys
# Add root directory to sys.path to import ExtractModule
sys.path.append(r"C:\Users\10552\OneDrive - Redlen Technologies\Code\H3D-app-deploy")
from Extract_module import ExtractModule, TransformDf
from icecream import ic
from spectrum_peak_finder import PeakFinder

print("Hello, World!")

Hello, World!


In [190]:
import os
def find_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [191]:
def crop_roi(array, crop_center, crop_halfwidth=40):
    bin_range = [crop_center - crop_halfwidth, crop_center + crop_halfwidth]
    cropped_array = array[bin_range[0]:bin_range[1]]
    return cropped_array

def find_peak_bin(array, crop_center, crop_halfwidth=40):
    cropped_array = crop_roi(array, crop_center, crop_halfwidth)
    bin_max = np.argmax(cropped_array) + crop_center - crop_halfwidth
    return bin_max

def find_peak_height(array, crop_center, crop_halfwidth=40):
    cropped_array = crop_roi(array, crop_center, crop_halfwidth)
    peak_height = np.max(cropped_array)
    return peak_height

In [192]:
def calculate_peak_count(array, peak_bin, peak_halfwidth=25):
    peak_count = np.sum(array[peak_bin - peak_halfwidth : peak_bin + peak_halfwidth])
    return peak_count

In [193]:
def extract_voltage(filename: str) -> int:
    string_parts = filename.split('_')[0]
    string_parts = string_parts.split('-')
    voltage = int(string_parts[2].replace('V', ''))
    return voltage

In [194]:
def avg_neighbor_counts(df, x_index, y_index, count_type='peak_count'):
    sum_counts = 0
    neighbor_coords = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    neighbor_counter = 0
    for dx, dy in neighbor_coords:
        nx, ny = x_index + dx, y_index + dy
        if (nx in df['x_index'].values) and (ny in df['y_index'].values):
            neighbor_counter += 1
            sum_counts += df.loc[(df['x_index']==nx) & (df['y_index']==ny), count_type].values[0]
    avg_counts = sum_counts / neighbor_counter
    return avg_counts

In [195]:
# def leaking_ratio(row):
#     return row['peak_count'] / row['avg_neighbor_peak_count']

In [196]:
source_peak_bins = {
    "Cs137": 1578,
    "Co57": 244,
    "Am241": 57
}

In [197]:
directory = r"module_voltage_data"
csv_files = find_csv_files(directory)
# print(csv_files)
df_list = []
for csv_file in csv_files:
    filename = os.path.basename(csv_file)
    filename_no_ext = os.path.splitext(filename)[0]
    print(f"{filename_no_ext = }")
    voltage = extract_voltage(filename)
    # print(f"{voltage = }")

    if "Cs137" in filename_no_ext:
        if "Co57" in filename_no_ext:
            source = "Co57"
        else:
            source = "Cs137"
    else:
        source = "Am241"

    EM = ExtractModule(csv_file)
    df_h3d = EM.extract_module2df(module_number=0)
    TD = TransformDf()
    df = TD.transform_df(df_h3d)

    df["source"] = source
    df["voltage"] = voltage
    df["is_treated"] = False
    df.loc[(df["x_index"] == 9) & (df["y_index"] == 3), "is_treated"] = True
    df.loc[(df["x_index"] == 3) & (df["y_index"] == 9), "is_treated"] = True
    df.loc[(df["x_index"] == 9) & (df["y_index"] == 9), "is_treated"] = True

    # # avg spectrum use to determine the bin_peak
    avg_array_bins = df["array_bins"].sum(axis=0) / 121
    avg_peak_bin= find_peak_bin(avg_array_bins, source_peak_bins[source])
    df['avg_peak_bin'] = avg_peak_bin
    df['avg_peak_height'] = find_peak_height(avg_array_bins, source_peak_bins[source])

    df['peak_bin_per_pixel'] = df.apply(lambda row: find_peak_bin(row['array_bins'], 
                                                        source_peak_bins[source]), axis=1)
    
    df['peak_height_per_pixel'] = df.apply(lambda row: find_peak_height(row['array_bins'], 
                                                        source_peak_bins[source]), axis=1)

    df = TD.add_peak_counts(df, bin_peak=avg_peak_bin, bin_width=25)

    df["avg_neighbor_peak_count"] = df.apply(
        lambda row: avg_neighbor_counts(
            df, row["x_index"], row["y_index"], "peak_count"
        ),
        axis=1,
    )

    df["leaking_ratio"] = df.apply(
        lambda row: row["peak_count"] / row["avg_neighbor_peak_count"], axis=1
    )

    df_list.append(df)


filename_no_ext = 'Am241-30min-1000V_Am241'
filename_no_ext = 'Am241-30min-1500V_Am241'
filename_no_ext = 'Am241-30min-2000V_Am241'
filename_no_ext = 'Co57-30min-1000V_Cs137'
filename_no_ext = 'Co57-30min-1500V_Cs137'
filename_no_ext = 'Co57-30min-2000V_Cs137'
filename_no_ext = 'Cs137-30min-1000V_Cs137'
filename_no_ext = 'Cs137-30min-1500V_Cs137'
filename_no_ext = 'Cs137-30min-2000V_Cs137'


In [198]:
# df['array_bins'].values
row = df.iloc[120]
print(row['array_bins'].shape)
print(row['array_bins'])
print(find_peak_bin(row['array_bins'], 97))
print(find_peak_height(row['array_bins'], 97))


(2002,)
[ 0  0  0 ...  0 11 11]
135
11


In [199]:
df['peak_bin'] = df.apply(lambda row: find_peak_bin(row['array_bins'], 
                                                    source_peak_bins[source]), axis=1)

print(df.head())

           x_index  y_index  pixel_id  \
H3D_Pixel                               
1                1        1         1   
2                1        2         2   
3                1        3         3   
4                1        4         4   
5                1        5         5   

                                                  array_bins  total_count  \
H3D_Pixel                                                                   
1          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...         5981   
2          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...         3722   
3          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...         3619   
4          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...         3483   
5          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...         4346   

           total_counts_norm  is_edge source  voltage  is_treated  \
H3D_Pixel                                                           
1                      0.992     T

In [200]:
full_df_2 = pd.concat(df_list)

In [201]:
# print(full_df.head())
print(full_df_2.columns)
print(full_df_2.shape)
print(full_df_2.info())

Index(['x_index', 'y_index', 'pixel_id', 'array_bins', 'total_count',
       'total_counts_norm', 'is_edge', 'source', 'voltage', 'is_treated',
       'avg_peak_bin', 'avg_peak_height', 'peak_bin_per_pixel',
       'peak_height_per_pixel', 'peak_count', 'non_peak_count',
       'avg_neighbor_peak_count', 'leaking_ratio', 'peak_bin'],
      dtype='object')
(1089, 19)
<class 'pandas.core.frame.DataFrame'>
Index: 1089 entries, 1 to 121
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   x_index                  1089 non-null   int32  
 1   y_index                  1089 non-null   int32  
 2   pixel_id                 1089 non-null   int64  
 3   array_bins               1089 non-null   object 
 4   total_count              1089 non-null   int32  
 5   total_counts_norm        1089 non-null   float64
 6   is_edge                  1089 non-null   bool   
 7   source                   1089 non-null 

In [202]:
# remove 'array_bins' column
full_df_2 = full_df_2.drop(columns=['array_bins'])

In [203]:
full_df_2.to_csv("full_df_2.csv", index=False)

In [204]:
full_df_2.to_csv("full_df.csv", index=False)